In [1]:
import json
from pathlib import Path
# file not provided, but it holds the authors from the cluster first middle and last
results = json.loads(Path("cluster_name_search.json").read_text())

In [2]:
results[0]

{'name': {'first': 'Vanessa', 'middle': 'Danielle', 'last': 'Hofschneider'},
 'result': {'meta': {'count': 0,
   'db_response_time_ms': 174,
   'page': 1,
   'per_page': 25},
  'results': [],
  'group_by': []}}

In [4]:
# attempt to see how many works were found by this author
non_zeros = [r for r in results if r["result"]["meta"]["count"]>0]

In [9]:
# maek a concept tree , and see how many counts we have per field
concept_counts = {}
for e in non_zeros:
    for person in e["result"]["results"]:
        concepts=person["x_concepts"]
        for concept in concepts:
            field_name = concept["display_name"]
            if not concept_counts.get(field_name,False):
                concept_counts[field_name]=1
            else:
                concept_counts[field_name]+=1

In [12]:

[e for e in concept_counts.items() if "humanities" in e[0].lower()]

[('Humanities', 335)]

In [20]:
# find the people associated with certain concepts
concept_counts = {}
affiliates = []
for e in non_zeros:
    for person in e["result"]["results"]:
        concepts=person["x_concepts"]
        for concept in concepts:
            field_name = concept["display_name"]
            if "humanities" in field_name.lower():
                affiliates.append({"name":e["name"],"person":person})

In [21]:
affiliates[0]

{'name': {'first': 'Adrian', 'middle': 'S', 'last': 'Martinez'},
 'person': {'id': 'https://openalex.org/A5005042949',
  'orcid': None,
  'display_name': 'Silvia Adrián Martínez',
  'display_name_alternatives': ['Silvia Adrián Martínez',
   'Silvia Martinez',
   'S. A. Martínez',
   'S. Martinez'],
  'relevance_score': 37.803257,
  'works_count': 7,
  'cited_by_count': 5,
  'summary_stats': {'2yr_mean_citedness': 0.0, 'h_index': 2, 'i10_index': 0},
  'ids': {'openalex': 'https://openalex.org/A5005042949'},
  'last_known_institution': {'id': 'https://openalex.org/I4210094364',
   'ror': 'https://ror.org/00smn7825',
   'display_name': 'Dubna State University',
   'country_code': 'RU',
   'type': 'education',
   'lineage': ['https://openalex.org/I4210094364']},
  'x_concepts': [{'id': 'https://openalex.org/C121332964',
    'wikidata': 'https://www.wikidata.org/wiki/Q413',
    'display_name': 'Physics',
    'level': 0,
    'score': 85.7},
   {'id': 'https://openalex.org/C62520636',
    'wi

In [17]:
# sanity check,go and get the pdfs and look for the institutional affiliations 
import requests as rq
url = affiliates[0]["works_api_url"]
res = rq.get(url)
works = res.json()

In [29]:
works["meta"]

{'count': 7, 'db_response_time_ms': 25, 'page': 1, 'per_page': 25}

In [28]:
works["meta"]["count"]

7

In [32]:
affiliates[0]["works_

{'name': {'first': 'Adrian', 'middle': 'S', 'last': 'Martinez'},
 'person': {'id': 'https://openalex.org/A5005042949',
  'orcid': None,
  'display_name': 'Silvia Adrián Martínez',
  'display_name_alternatives': ['Silvia Adrián Martínez',
   'Silvia Martinez',
   'S. A. Martínez',
   'S. Martinez'],
  'relevance_score': 37.803257,
  'works_count': 7,
  'cited_by_count': 5,
  'summary_stats': {'2yr_mean_citedness': 0.0, 'h_index': 2, 'i10_index': 0},
  'ids': {'openalex': 'https://openalex.org/A5005042949'},
  'last_known_institution': {'id': 'https://openalex.org/I4210094364',
   'ror': 'https://ror.org/00smn7825',
   'display_name': 'Dubna State University',
   'country_code': 'RU',
   'type': 'education',
   'lineage': ['https://openalex.org/I4210094364']},
  'x_concepts': [{'id': 'https://openalex.org/C121332964',
    'wikidata': 'https://www.wikidata.org/wiki/Q413',
    'display_name': 'Physics',
    'level': 0,
    'score': 85.7},
   {'id': 'https://openalex.org/C62520636',
    'wi

In [35]:
# collect institutional works based on a affiliation string containing the word arizona
ua_works = []
import time
import random
for aff in affiliates:
    time.sleep(random.random()*.5)
    aff = aff["person"]
    url = aff["works_api_url"]
    res = rq.get(url)
    works = res.json()
    count = works["meta"]["count"]
    per_page = works["meta"]["per_page"]
    print("within page limit",count < per_page)
    for work in works["results"]:
        # check if the count is greater than the page size
        # print(work)
        for author in work["authorships"]:
            # check the affiliation string
            if author.get("raw_affiliation_string"):
                if "arizona" in author.get("raw_affiliation_string"):
                    ua_works.append(work)

within page limit True
within page limit False
within page limit False
within page limit True
within page limit True
within page limit True
within page limit True
within page limit True
within page limit True
within page limit False
within page limit True
within page limit True
within page limit True
within page limit False
within page limit True
within page limit True
within page limit True
within page limit True
within page limit True
within page limit True
within page limit True
within page limit True
within page limit True
within page limit True
within page limit True
within page limit True
within page limit True
within page limit True
within page limit True
within page limit False
within page limit False
within page limit False
within page limit False
within page limit False
within page limit True
within page limit True
within page limit True
within page limit True
within page limit False
within page limit True
within page limit True
within page limit True
within page limit True
w

KeyboardInterrupt: 

In [19]:
url


'https://api.openalex.org/works?filter=author.id:A5005042949'

In [36]:
ua_works

[]

https://api.openalex.org/authors?filter=last_known_institution.id:I138006243 this is the query for the u of arizona alone

perhaps from this we could get all the names, and then only match to the ones that are part of the HPC users name set


https://api.openalex.org/authors?filter=last_known_institution.id:I138006243,x_concepts.id:C15708023 gets all the present authors from UA in humanities 

In [39]:
# attempt to put the humanities specific filter in the openalex query itself
humanities =[]
for i in range(2):
    res = rq.get(f"https://api.openalex.org/authors?filter=last_known_institution.id:I138006243,x_concepts.id:C15708023&per-page=200&page={i+1}")
    humanities.append(res.json()["results"])

In [40]:
humanities

[[{'id': 'https://openalex.org/A5069825706',
   'orcid': 'https://orcid.org/0000-0002-3878-319X',
   'display_name': 'Albrecht Classen',
   'display_name_alternatives': ['Albrecht Classen', 'A. Classen'],
   'works_count': 1638,
   'cited_by_count': 993,
   'summary_stats': {'2yr_mean_citedness': 0.09565217391304348,
    'h_index': 11,
    'i10_index': 13},
   'ids': {'openalex': 'https://openalex.org/A5069825706',
    'orcid': 'https://orcid.org/0000-0002-3878-319X'},
   'last_known_institution': {'id': 'https://openalex.org/I138006243',
    'ror': 'https://ror.org/03m2x1q45',
    'display_name': 'University of Arizona',
    'country_code': 'US',
    'type': 'education',
    'lineage': ['https://openalex.org/I138006243',
     'https://openalex.org/I3132585960']},
   'x_concepts': [{'id': 'https://openalex.org/C142362112',
     'wikidata': 'https://www.wikidata.org/wiki/Q735',
     'display_name': 'Art',
     'level': 0,
     'score': 76.0},
    {'id': 'https://openalex.org/C138885662'

In [41]:
all_names = json.loads(Path("cluster_names.json").read_text())

In [43]:
humanities_flat = [hum for g in humanities for hum in g]

In [47]:
humanities_flat[0]["display_name"]

'Albrecht Classen'

In [54]:
hits= []
for e in humanities_flat:
    for a in all_names:
        if a["first"] in e["display_name"] and a["last"] in e["display_name"]:
            hits.append(e)
            break

In [55]:
len(hits)

1

In [56]:
# not good, only one hit
hits

[{'id': 'https://openalex.org/A5020821908',
  'orcid': 'https://orcid.org/0000-0001-7576-5153',
  'display_name': 'Sanho Chung',
  'display_name_alternatives': ['Sanho Chung'],
  'works_count': 3,
  'cited_by_count': 28,
  'summary_stats': {'2yr_mean_citedness': 0.0, 'h_index': 1, 'i10_index': 1},
  'ids': {'openalex': 'https://openalex.org/A5020821908',
   'orcid': 'https://orcid.org/0000-0001-7576-5153'},
  'last_known_institution': {'id': 'https://openalex.org/I138006243',
   'ror': 'https://ror.org/03m2x1q45',
   'display_name': 'University of Arizona',
   'country_code': 'US',
   'type': 'education',
   'lineage': ['https://openalex.org/I138006243',
    'https://openalex.org/I3132585960']},
  'x_concepts': [{'id': 'https://openalex.org/C17744445',
    'wikidata': 'https://www.wikidata.org/wiki/Q36442',
    'display_name': 'Political science',
    'level': 0,
    'score': 100.0},
   {'id': 'https://openalex.org/C94625758',
    'wikidata': 'https://www.wikidata.org/wiki/Q7163',
    

In [67]:
# see if we can start looking for people not currently at the u of a
past_ua_folks = rq.get("https://api.openalex.org/authors?filter=last_known_institution.lineage:I138006243,last_known_institution.id:!I138006243").json()
present_ua_folks = rq.get("https://api.openalex.org/authors?filter=last_known_institution.lineage:I138006243,last_known_institution.id:I138006243").json()


In [64]:
# see if any had a latest lineage value different from I138006243
lin = "I138006243"
for res in past_ua_folks["results"]:
    last_institution = res["last_known_institution"]
    lineage = last_institution["lineage"]
    display_name = last_institution["display_name"]
    if not lin in lineage[0] :
        print(res["display_name"])
        print(display_name)
        print(lineage)
        print()

In [81]:
import math

In [98]:
# add researchers to a list whetherthey are currently at u of arizona or not
all_ua_researchers = []
keep_going = True
page=1
print("first half")
while keep_going:
    past_ua_folks = rq.get(f"https://api.openalex.org/authors?filter=last_known_institution.lineage:I138006243,last_known_institution.id:!I138006243&page={page}&per-page=200").json()
    count = past_ua_folks["meta"]["count"]
    per_page = past_ua_folks["meta"]["per_page"]
    pages_tot = math.ceil(count/per_page)
    print(f"page {page} of {pages_tot}")
    page+=1
    
    all_ua_researchers.extend(past_ua_folks["results"])
    if page > pages_tot:
        break
    time.sleep(random.random()*.5)
page=1
print("second_half\n")
start = rq.get(f"https://api.openalex.org/authors?filter=last_known_institution.lineage:I138006243,last_known_institution.id:I138006243&cursor=*&per-page=200").json()
cursor_token= start["meta"]["next_cursor"]
records = 0
while cursor_token:
    present_ua_folks = rq.get(f"https://api.openalex.org/authors?filter=last_known_institution.lineage:I138006243,last_known_institution.id:I138006243&cursor={cursor_token}&per-page=200").json()
    all_ua_researchers.extend(present_ua_folks["results"])
    cursor_token = present_ua_folks["meta"]["next_cursor"]
    records+=200
    print("records",records)
    if len(present_ua_folks["results"]) ==0:
        break
    time.sleep(random.random()*.5)


first half
page 1 of 1
second_half

records 200
records 400
records 600
records 800
records 1000
records 1200
records 1400
records 1600
records 1800
records 2000
records 2200
records 2400
records 2600
records 2800
records 3000
records 3200
records 3400
records 3600
records 3800
records 4000
records 4200
records 4400
records 4600
records 4800
records 5000
records 5200
records 5400
records 5600
records 5800
records 6000
records 6200
records 6400
records 6600
records 6800
records 7000
records 7200
records 7400
records 7600
records 7800
records 8000
records 8200
records 8400
records 8600
records 8800
records 9000
records 9200
records 9400
records 9600
records 9800
records 10000
records 10200
records 10400
records 10600
records 10800
records 11000
records 11200
records 11400
records 11600
records 11800
records 12000
records 12200
records 12400
records 12600
records 12800
records 13000
records 13200
records 13400
records 13600
records 13800
records 14000
records 14200
records 14400
records 1

In [99]:
Path("all_ua_authors.json").write_text(json.dumps(all_ua_researchers))

118507333

In [83]:
past_ua_folks

{'meta': {'count': 9, 'db_response_time_ms': 60, 'page': 1, 'per_page': 200},
 'results': [{'id': 'https://openalex.org/A5091612755',
   'orcid': None,
   'display_name': 'Charles H. Tweed',
   'display_name_alternatives': ['C H Tweed', 'Charles H. Tweed'],
   'works_count': 14,
   'cited_by_count': 978,
   'summary_stats': {'2yr_mean_citedness': 0.0,
    'h_index': 10,
    'i10_index': 10},
   'ids': {'openalex': 'https://openalex.org/A5091612755'},
   'last_known_institution': {'id': 'https://openalex.org/I4210143680',
    'ror': 'https://ror.org/054zw2979',
    'display_name': 'Arizona Space Grant Consortium',
    'country_code': 'US',
    'type': 'other',
    'lineage': ['https://openalex.org/I138006243',
     'https://openalex.org/I4210124779',
     'https://openalex.org/I4210143680']},
   'x_concepts': [{'id': 'https://openalex.org/C29694066',
     'wikidata': 'https://www.wikidata.org/wiki/Q118301',
     'display_name': 'Orthodontics',
     'level': 1,
     'score': 100.0},
    

In [78]:
page

1

In [85]:
# start experimenting with paging
f"https://api.openalex.org/authors?filter=last_known_institution.lineage:I138006243,last_known_institution.id:!I138006243&page={page}&per-page=200"

'https://api.openalex.org/authors?filter=last_known_institution.lineage:I138006243,last_known_institution.id:!I138006243&page=51&per-page=200'

In [86]:
count

26899

In [88]:
past_ua_folks = rq.get(f"https://api.openalex.org/authors?filter=last_known_institution.lineage:I138006243,last_known_institution.id:!I138006243&page={1}&per-page=200").json()


In [93]:
past_ua_folks

{'meta': {'count': 9, 'db_response_time_ms': 72, 'page': 1, 'per_page': 200},
 'results': [{'id': 'https://openalex.org/A5091612755',
   'orcid': None,
   'display_name': 'Charles H. Tweed',
   'display_name_alternatives': ['C H Tweed', 'Charles H. Tweed'],
   'works_count': 14,
   'cited_by_count': 978,
   'summary_stats': {'2yr_mean_citedness': 0.0,
    'h_index': 10,
    'i10_index': 10},
   'ids': {'openalex': 'https://openalex.org/A5091612755'},
   'last_known_institution': {'id': 'https://openalex.org/I4210143680',
    'ror': 'https://ror.org/054zw2979',
    'display_name': 'Arizona Space Grant Consortium',
    'country_code': 'US',
    'type': 'other',
    'lineage': ['https://openalex.org/I138006243',
     'https://openalex.org/I4210124779',
     'https://openalex.org/I4210143680']},
   'x_concepts': [{'id': 'https://openalex.org/C29694066',
     'wikidata': 'https://www.wikidata.org/wiki/Q118301',
     'display_name': 'Orthodontics',
     'level': 1,
     'score': 100.0},
    

In [94]:
past_ua_folks["meta"]["count"]/past_ua_folks["meta"]["per_page"]

0.045

In [95]:
math.ceil(past_ua_folks["meta"]["count"]/past_ua_folks["meta"]["per_page"])

1